In [ ]:
import platform, torch
print("OS:", platform.platform())
print("PyTorch:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)
print("GPU count:", torch.cuda.device_count())
print("MPS available:", getattr(torch.backends, "mps", None) and torch.backends.mps.is_available())
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Device 0:", torch.cuda.get_device_name(0))
    print("CUDA runtime:", torch.version.cuda)

OS: Windows-11-10.0.26200-SP0
PyTorch: 2.8.0+cu129
torch.version.cuda: 12.9
GPU count: 1
MPS available: False
CUDA available: True
CUDA device count: 1
Device 0: NVIDIA GeForce RTX 5080
CUDA runtime: 12.9


In [6]:
import os
import sys
import shutil
import random
import torch

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

from helper import count_labels_per_class

In [7]:
random.seed(42)
torch.manual_seed(42)

In [8]:
def split_dataset(img_dir, label_dir, output_img_dir, output_label_dir, train_ratio=0.8):
    """
    Create a stratified train/val split based on fruit names inferred from filenames or from a classes.txt file.

    Approach:
    - If `classes.txt` exists in `label_dir`, use those class names and match them (case-insensitive) against filenames to assign a class to each image.
    - Otherwise fall back to underscore/token parsing to pick a likely fruit token from the filename.
    - For each class/group, shuffle and split locally to match `train_ratio` (so each fruit is balanced across splits).
    - Copy corresponding image and .txt label (if present) into output split dirs under `train` and `val`.
    """
    import re

    # read image files
    img_files = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not img_files:
        print('No images found in', img_dir); return

    # try to read classes.txt to know fruit names (optional)
    classes_path = os.path.join(label_dir, 'classes.txt')
    classes = []
    if os.path.exists(classes_path):
        try:
            with open(classes_path, 'r', encoding='utf-8') as f:
                classes = [line.strip() for line in f if line.strip()]
        except Exception as e:
            print('Warning: could not read classes.txt:', e)
            classes = []

    # helper to infer class/fruit from filename
    def infer_class_from_name(name):
        base = os.path.splitext(name)[0]
        lower = base.lower()
        # 1) match any class token from classes.txt
        for c in classes:
            if c and c.lower() in lower:
                return c
        # 2) fallback: split on non-alphanum and try tokens
        tokens = re.split(r'[_\W]+', base)
        # prefer tokens that are purely alphabetic and longer than 1 char
        for t in tokens:
            if not t:
                continue
            for c in classes:
                if t.lower() == c.lower():
                    return c
        # 3) if no classes.txt or no match, heuristically pick the token that looks like a fruit (first alphabetic token)
        for t in tokens:
            if t.isalpha():
                return t
        # 4) final fallback
        return 'unknown'

    # group images by inferred class
    groups = {}
    for fn in img_files:
        cls = infer_class_from_name(fn)
        groups.setdefault(cls, []).append(fn)

    # build train/val lists by splitting each group independently
    train_imgs = []
    val_imgs = []
    for cls, files in groups.items():
        random.shuffle(files)
        n_train = int(len(files) * train_ratio)
        # ensure at least one example in train or val when possible
        if n_train == 0 and len(files) > 1:
            n_train = 1
        train_imgs.extend(files[:n_train])
        val_imgs.extend(files[n_train:])
        print(f'Class {cls}: total={len(files)}, train={n_train}, val={len(files)-n_train}')

    # Helper: copy files to new split directories
    def copy_files(img_list, split):
        split_img_dir = os.path.join(output_img_dir, split)
        split_label_dir = os.path.join(output_label_dir, split)
        os.makedirs(split_img_dir, exist_ok=True)
        os.makedirs(split_label_dir, exist_ok=True)

        for img_name in img_list:
            base_name = os.path.splitext(img_name)[0]
            src_img = os.path.join(img_dir, img_name)
            src_label = os.path.join(label_dir, base_name + '.txt')
            dst_img = os.path.join(split_img_dir, img_name)
            dst_label = os.path.join(split_label_dir, base_name + '.txt')
            try:
                shutil.copy(src_img, dst_img)
            except Exception as e:
                print('Failed copying image', src_img, '->', dst_img, e)
            if os.path.exists(src_label):
                try:
                    shutil.copy(src_label, dst_label)
                except Exception as e:
                    print('Failed copying label', src_label, '->', dst_label, e)

    # perform copies
    copy_files(train_imgs, 'train')
    copy_files(val_imgs, 'val')

    print(f'Total images: {len(img_files)}; Train: {len(train_imgs)}; Val: {len(val_imgs)}')

In [10]:
img_dir = "../dataset/input/images"
label_dir = "../dataset/input/labels"
output_img_dir = "../dataset/split/images"
output_label_dir = "../dataset/split/labels"
split_dataset(img_dir, label_dir, output_img_dir, output_label_dir, train_ratio=0.8)


Class img: total=1354, train=1083, val=271
Total images: 1354; Train: 1083; Val: 271


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # load a pretrained YOLOv8n model
model.train(
    data = "../data.yaml",
    epochs = 20,
    imgsz = 640,
    batch = 16,
    device = device
)


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Admin\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mix

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000023CA2C00550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,   

In [14]:
results = model("../data/test_images/fruitsrec_Apple_Apple 0444.png", conf=0.1)
boxes = results[0].boxes.xyxy  # Detected bounding boxes
print(boxes)

FileNotFoundError: ../data/test_images/fruitsrec_Apple_Apple 0444.png does not exist